# Data Cleaning




## Objective

The objective of this stage is to improve data quality and consistency across the Airbnb datasets for Berlin and Bangkok. This includes identifying and addressing missing values, duplicates, formatting inconsistencies, and invalid records to ensure the data is reliable and suitable for exploratory analysis and dashboard development.



## Data Quality Issues Identified

* Missing values are present in both numerical fields (such as price and reviews per month) and categorical fields (such as host name and license). These can affect statistical calculations and category-based analysis, so each affected column must be handled carefully during data cleaning.

* Price-related fields require validation to ensure values are numeric and within realistic ranges. Extremely high, zero, or incorrectly formatted prices can distort average price calculations and lead to misleading comparisons between cities.

* There may be duplicate listings based on listing identifiers, which can inflate property counts and review statistics. Verifying and removing duplicates is necessary to ensure each property is represented only once.

* Text-based fields like neighbourhood and room type may contain inconsistent formatting (such as extra spaces or case differences), which can split the same category into multiple groups and affect grouping and visualization accuracy.

* Review-related fields contain missing or sparse values, often indicating new or inactive listings rather than data errors. These should be interpreted carefully to avoid removing meaningful information about market activity.


## Cleaning Strategy



## Handling Missing Values



## Handling Duplicates



## Standardizing Formats



## Output Datasets